# Graphs (Agent Workflows)

In this section you'll learn how to create an _agent workflow_ using **AGGraph**. A workflow is a structured execution plan that precisely controls how agents interact to accomplish a task.

We'll first show you how to create and run a workflow. We'll then explain how to observe and debug workflow behavior, and discuss important operations for managing execution.

AutoGen provides a single workflow class for directed graph execution:

- {py:class}`~autogen_agentchat.teams.AGGraph`: A team that follows a **directed graph** to control the execution flow between agents. Supports sequential, parallel, conditional, and looping behaviors.

```{note}
**When should you use a workflow?**

Use AGGraph when you need strict control over the order in which agents act, or when different outcomes must lead to different next steps.
Start with a simple team if natural conversation is sufficient. Transition to a structured workflow when your task requires deterministic control,
conditional branching, or handling complex multi-step processes with cycles.
```

> **Warning:** AGGraph is an **experimental feature**. Its API, behavior, and capabilities are **subject to change** in future releases.

## Creating and runnng a Workflow
AGGraphBuilder is a fluent utility that lets you easily construct execution graphs for workflows. It supports building:

- Sequential chains
- Parallel fan-outs
- Conditional branching
- Loops with safe exit conditions

Each node in the graph represents an agent, and edges define the allowed execution paths. Edges can optionally have conditions based on agent messages.

### Sequential Workflow
We will begin by creating a simple workflow where a **writer** drafts a paragraph and a **reviewer** provides feedback. This graph terminates after the reviewer comments on the writer. 

Note, AGGraph automatically computes all the source and leaf nodes of the graph and the execution starts at all the source nodes in the graph and completes execution when no nodes are left to execute.

In [ ]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import AGGraph, AGGraphBuilder
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an OpenAI model client
client = OpenAIChatCompletionClient(model="gpt-4o-mini")

# Create the writer agent
writer = AssistantAgent("writer", model_client=client, system_message="Draft a short paragraph on climate change.")

# Create the reviewer agent
reviewer = AssistantAgent("reviewer", model_client=client, system_message="Review the draft and suggest improvements.")

# Build the workflow graph
builder = AGGraphBuilder()
builder.add_node(writer).add_node(reviewer)
builder.add_edge(writer, reviewer)

# Build and validate the graph
graph = builder.build()

# Create the workflow
workflow = AGGraph(participants=builder.get_participants(), graph=graph)

In [ ]:
# Use `asyncio.run(...)` when running in a script.

result = await workflow.run(task="Write a short paragraph about climate change.")
print(result)

### Parallel Workflow with Join

We now create a slightly more complex workflow:

- A **writer** drafts a paragraph.
- Two **editors** independently edit for grammar and style (parallel fan-out).
- A **final reviewer** consolidates their edits (join).

Execution starts at the **writer**, fans out to **editor1** and **editor2** simultaneously, and then both feed into the **final reviewer**.


In [2]:

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.teams import AGGraph, AGGraphBuilder
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an OpenAI model client
client = OpenAIChatCompletionClient(model="gpt-4o-mini")

# Create the writer agent
writer = AssistantAgent("writer", model_client=client, system_message="Draft a short paragraph on climate change.")

# Create two editor agents
editor1 = AssistantAgent("editor1", model_client=client, system_message="Edit the paragraph for grammar.")

editor2 = AssistantAgent("editor2", model_client=client, system_message="Edit the paragraph for style.")

# Create the final reviewer agent
final_reviewer = AssistantAgent(
    "final_reviewer",
    model_client=client,
    system_message="Consolidate the grammar and style edits into a final version.",
)

# Build the workflow graph
builder = AGGraphBuilder()
builder.add_node(writer).add_node(editor1).add_node(editor2).add_node(final_reviewer)

# Fan-out from writer to editor1 and editor2
builder.add_edge(writer, editor1)
builder.add_edge(writer, editor2)

# Fan-in both editors into final reviewer
builder.add_edge(editor1, final_reviewer)
builder.add_edge(editor2, final_reviewer)

# Build and validate the graph
graph = builder.build()

# Create the workflow
workflow = AGGraph(
    participants=builder.get_participants(),
    graph=graph,
    termination_condition=MaxMessageTermination(10),
)

# Run the workflow
result = await workflow.run(task="Write a short paragraph about climate change.")
print(result)

messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a short paragraph about climate change.', type='TextMessage'), TextMessage(source='writer', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=111), metadata={}, content='Climate change refers to long-term shifts and alterations in temperature and weather patterns, primarily triggered by human activities such as burning fossil fuels, deforestation, and industrial processes. The accumulation of greenhouse gases in the atmosphere has led to global warming, resulting in rising sea levels, extreme weather events, and disruptions to ecosystems. As the planet grapples with these challenges, urgent actions are necessary to mitigate impacts, adapt to changes, and transition towards sustainable practices. Addressing climate change not only requires global cooperation but also innovative solutions that prioritize environmental health and social equity.', type='TextMessage'), TextMessage(source='editor1

## Message Filtering

### Understanding Execution Graph vs. Message Graph
In AGGraph, the **execution graph** controls **which agents speak and in what order**.
However, it does not control **what messages an agent can see**.

**Message filtering** allows you to limit the context visible to each agent, even though the overall execution order is still controlled by the graph.
This can help:
- Reduce hallucinations
- Control memory load
- Focus agents only on relevant information

You can use {py:class}`~autogen_agentchat.teams.MessageFilterAgent` together with {py:class}`~autogen_agentchat.teams.MessageFilterConfig` and {py:class}`~autogen_agentchat.teams.PerSourceFilter` to define these rules.

In [5]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import AGGraph, AGGraphBuilder, MessageFilterAgent, MessageFilterConfig, PerSourceFilter
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Model client
client = OpenAIChatCompletionClient(model="gpt-4o-mini")

# Create agents
researcher = AssistantAgent(
    "researcher", model_client=client, system_message="Summarize key facts about climate change."
)

analyst = AssistantAgent("analyst", model_client=client, system_message="Review the summary and suggest improvements.")

presenter = AssistantAgent(
    "presenter", model_client=client, system_message="Prepare a presentation slide based on the final summary."
)

# Apply message filtering
filtered_analyst = MessageFilterAgent(
    name="analyst",
    wrapped_agent=analyst,
    filter=MessageFilterConfig(per_source=[PerSourceFilter(source="researcher", position="last", count=1)]),
)

filtered_presenter = MessageFilterAgent(
    name="presenter",
    wrapped_agent=presenter,
    filter=MessageFilterConfig(per_source=[PerSourceFilter(source="analyst", position="last", count=1)]),
)

# Build the workflow
builder = AGGraphBuilder()
builder.add_node(researcher).add_node(filtered_analyst).add_node(filtered_presenter)
builder.add_edge(researcher, filtered_analyst).add_edge(filtered_analyst, filtered_presenter)

# Create the workflow
workflow = AGGraph(
    participants=builder.get_participants(),
    graph=builder.build(),
    termination_condition=MaxMessageTermination(10),
)

# Run the workflow
result = await workflow.run(task="Summarize key facts about climate change.")
print(result)

messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Summarize key facts about climate change.', type='TextMessage'), TextMessage(source='researcher', models_usage=RequestUsage(prompt_tokens=30, completion_tokens=480), metadata={}, content="1. **Definition**: Climate change refers to significant and lasting changes in the Earth's climate, primarily due to human activities, including increased concentrations of greenhouse gases (GHGs) in the atmosphere.\n\n2. **Greenhouse Gases**: The main GHGs contributing to climate change are carbon dioxide (CO2), methane (CH4), and nitrous oxide (N2O). These gases trap heat in the atmosphere and lead to global warming.\n\n3. **Temperature Rise**: The Earth's average temperature has risen by about 1.2 degrees Celsius (2.2 degrees Fahrenheit) since the late 19th century, largely due to increased CO2 emissions from burning fossil fuels.\n\n4. **Climate Impact**: Climate change causes extreme weather events, such as hurricanes, 

## Core Concepts

{py:class}`~autogen_agentchat.teams.AGGraph` introduces several important building blocks that work together to create directed agent workflows.

### {py:class}`~autogen_agentchat.teams.AGGraphBuilder`
The {py:class}`~autogen_agentchat.teams.AGGraphBuilder` is a **fluent builder** for constructing execution graphs programmatically.
It provides methods to:
- Add agents (nodes) to the graph.
- Add edges between agents, optionally with conditions.
- Define complex topologies including parallel, conditional, and looping flows.

Typical usage:
```python
builder = AGGraphBuilder()
builder.add_node(agent_a).add_node(agent_b)
builder.add_edge(agent_a, agent_b)
graph = builder.build()
```

{py:class}`~autogen_agentchat.teams.AGGraphBuilder` automatically validates that the graph structure is sound before allowing execution.


### DiGraphNode
Represents an agent node in the directed graph.

Fields:
- `name`: The unique name of the agent.
- `edges`: A list of outgoing edges (to other nodes).
- `activation`: Either "all" (wait for all parents to finish) or "any" (activate when any parent finishes).

Example:
```python
DiGraphNode(name="writer", edges=[DiGraphEdge(target="reviewer")], activation="all")
```


### DiGraphEdge
Represents a directed connection from one node to another.

Fields:
- `target`: The name of the target node.
- `condition`: (Optional) If set, the transition only happens if the message content matches the condition.

Example:
```python
DiGraphEdge(target="reviewer", condition="approve")
```


### DiGraph
A validated collection of DiGraphNode objects forming the execution graph.

Key features:
- Computes **start nodes** (nodes with no incoming edges).
- Computes **leaf nodes** (nodes with no outgoing edges).
- Validates that any cycles in the graph have safe exits (i.e., conditional branches).

A DiGraph ensures the workflow has at least one entry point and at least one termination point.


### {py:class}`~autogen_agentchat.teams.AGGraph`
The main runner class that:
- Executes agents according to the DiGraph structure.
- Manages agent selection at each turn.
- Supports saving and restoring execution state (e.g., for retries, continuation).
- Supports flexible termination conditions (e.g., max turns, text triggers).

Example:
```python
workflow = AGGraph(participants=builder.get_participants(), graph=builder.build())
await workflow.run(task="Draft and review a document.")
```

{py:class}`~autogen_agentchat.teams.AGGraph` separates **execution control** (who speaks when) from **message filtering** (what each agent can see).


### Message Filtering
Even though {py:class}`~autogen_agentchat.teams.AGGraph` controls agent execution order, agents may still have access to previous conversation history. 

To **control what messages an agent sees**, you can:
- Wrap the agent using {py:class}`~autogen_graph.MessageFilterAgent`.
- Specify fine-grained rules using {py:class}`~autogen_graph.MessageFilterConfig` and {py:class}`~autogen_graph.PerSourceFilter`.

This allows focusing an agent’s attention only on:
- Specific messages from certain sources.
- Only the most recent (or oldest) messages.

Example:
```python
filtered_agent = MessageFilterAgent(
    name="critic",
    wrapped_agent=critic,
    filter=MessageFilterConfig(per_source=[
        PerSourceFilter(source="writer", position="last", count=1)
    ])
)
```

### Activation Modes
Each node can have an activation mode:
- **all** (default): The agent activates after **all** its parent nodes have finished.
- **any**: The agent activates after **any one** parent node finishes.

This allows modeling different coordination behaviors:
- Synchronization points (all parents must complete).
- Early branching (first parent to finish triggers execution).

### Conditional Branching
Edges can be **conditional**: only followed if a specific text condition is matched in the message.

Example:
```python
builder.add_conditional_edges(agent_a, {"yes": agent_b, "no": agent_c})
```
This enables modeling decision trees, approvals, rejections, and loops.


### Loops with Safe Exit
{py:class}`~autogen_agentchat.teams.AGGraph` allows loops, but **only** if:
- There is at least one conditional edge that can break the cycle.
- Cycles without exit conditions are rejected at graph validation time.

This ensures no infinite loops occur during execution.

## 🔁 Advanced Example: Conditional Loop + Filtered Summary

This example demonstrates:

- A loop between generator and reviewer (which exits when reviewer says "final")
- A summarizer agent that only sees the first user input and the last reviewer message


In [9]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import (
    AGGraph,
    AGGraphBuilder,
    MessageFilterAgent,
    MessageFilterConfig,
    PerSourceFilter,
)
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(model="gpt-4o")

# Agents
generator = AssistantAgent("generator", model_client=model_client, system_message="Generate a list of creative ideas.")
reviewer = AssistantAgent("reviewer", model_client=model_client, system_message="Review ideas and say LOOP or FINAL.")
summarizer_core = AssistantAgent(
    "summary", model_client=model_client, system_message="Summarize the user request and the final feedback."
)

# Filtered summarizer
filtered_summarizer = MessageFilterAgent(
    name="summary",
    wrapped_agent=summarizer_core,
    filter=MessageFilterConfig(
        per_source=[
            PerSourceFilter(source="user", position="first", count=1),
            PerSourceFilter(source="reviewer", position="last", count=1),
        ]
    ),
)

# Build graph with conditional loop
builder = AGGraphBuilder()
builder.add_node(generator).add_node(reviewer).add_node(filtered_summarizer)
builder.add_edge(generator, reviewer)
builder.add_edge(reviewer, generator, condition="LOOP")
builder.add_edge(reviewer, filtered_summarizer, condition="FINAL")
builder.set_entry_point(generator)  # Set entry point to generator. Required if there are no source nodes.

workflow = AGGraph(
    participants=builder.get_participants(),
    graph=builder.build(),
    termination_condition=TextMentionTermination("FINAL"),
)
result = await workflow.run(task="Brainstorm ways to reduce plastic waste.")
print(result)

messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Brainstorm ways to reduce plastic waste.', type='TextMessage'), TextMessage(source='generator', models_usage=RequestUsage(prompt_tokens=27, completion_tokens=559), metadata={}, content='1. **Biodegradable Alternatives**: Develop and promote the use of biodegradable materials for packaging, such as plant-based plastics, mushroom-based packaging, or seaweed-derived materials.\n\n2. **Reusable Packaging Systems**: Implement a system where consumers can return and refill containers for products like shampoo, detergent, and cleaning supplies at local stores, similar to milkmen systems.\n\n3. **Plastic Waste Turned Products**: Encourage businesses to create products out of recycled plastic such as home decor items, furniture, or fashion accessories, which can be made from ocean plastic waste.\n\n4. **Incentive Programs**: Start community programs that offer rewards, like discounts or charitable donations, in exchan